# STOR 320 Introduction to Data Science

## Week 10: Cross validation

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Helper Functions

In [10]:
def OSR2(y_train, y_test, y_pred):
    
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

In [11]:
def MAE(y_test, y_pred):
    
    return (np.mean(abs(y_test - y_pred)))

In [12]:
def RMSE(y_test, y_pred):
    
    return np.sqrt(np.mean((y_test - y_pred)**2))

In [13]:
def print_metrics(model, X_train, y_train, X_test, y_test, flag_log_sale_price=False):

    if (flag_log_sale_price == True):
        
        y_pred_train = pd.Series(model.predict(X_train)).reset_index(drop=True)
        y_pred_test = pd.Series(model.predict(X_test)).reset_index(drop=True)
        y_train = y_train.copy().reset_index(drop=True)
        y_test = y_test.copy().reset_index(drop=True)
        
        print("\nMetrics for Log(Sale Price):\n")
        
    elif (flag_log_sale_price == False):
        
        y_pred_train = pd.Series(model.predict(X_train)).apply(np.exp).reset_index(drop=True)
        y_pred_test = pd.Series(model.predict(X_test)).apply(np.exp).reset_index(drop=True)
        y_train = y_train.copy().apply(np.exp).reset_index(drop=True)
        y_test = y_test.copy().apply(np.exp).reset_index(drop=True)
        
        print("\nMetrics for Sale Price:\n")

    print('Training R2', OSR2(y_train, y_train, y_pred_train))
    print('Training MAE', MAE(y_train, y_pred_train))
    print('Training RMSE', RMSE(y_train, y_pred_train))

    print('Out-of-sample R2', OSR2(y_train, y_test, y_pred_test))
    print('Out-of-sample MAE', MAE(y_test, y_pred_test))
    print('Out-of-sample RMSE', RMSE(y_test, y_pred_test))
    
    return None

In [14]:
ames = pd.read_csv('cleaned_Ames.csv')
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2765 entries, 0 to 2764
Columns: 105 entries, Unnamed: 0 to YearsSince1950GarageBuilt
dtypes: float64(21), int64(45), object(39)
memory usage: 2.2+ MB


In [15]:
ames = pd.read_feather('cleaned_ames.feather')
ames

ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

In [ ]:
ames.info()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ames_train = ames.loc[ames['YrSold'].isin([2006, 2007, 2008])]
ames_test = ames.loc[ames['YrSold'].isin([2009, 2010])]

ames = ames.drop(columns = ['YrSold'])
ames_train = ames_train.drop(columns = ['YrSold'])
ames_test = ames_test.drop(columns = ['YrSold'])

y_train = ames_train['LogSalePrice']
y_test = ames_test['LogSalePrice']

print(ames.shape, ames_train.shape, ames_test.shape)

### Simple Linear model with higher-order Variables


In [ ]:
def create_polynomial_features(df, n_degree):

    new_df = None
    
    for i in range(2, n_degree+1):
        
        tmp = df.pow(i)
        
        affix = '_p'+str(i)
        tmp.columns = list(map(lambda x: x + affix, df.columns))
        
        if new_df is not None:
            new_df = pd.concat([new_df, tmp], axis=1)
        else:
            new_df = tmp
    
    return new_df

<span style='color:blue'>
    
NOTE: An important consideration when creating higher-order variables is that the resulting features will tend to have some degree of linear dependence amongst themselves. This is normal as several new features are based on their zero-th power peer. Such correlation can also yield a high degree of multicollinearity in the regression models. The `sklearn` implementations that we will be using do not automatically account for this phenomenon, therefore we must be careful in selection the `n_degree`, and analyzing the model fit. 

</span>

In [ ]:
# We only choose a select list of variables to do polynomial transformation.
poly_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
             'X1stFlrSF', 'X2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
             'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch', 'MiscVal', 'YearsSince1950Built',
             'YearsSince1950Remod', 'YearsSince1950GarageBuilt']

In [ ]:
n_degree = 2

train_poly_temp = create_polynomial_features(ames_train[poly_cols], n_degree)
test_poly_temp = create_polynomial_features(ames_test[poly_cols], n_degree)

ames_train_poly = pd.concat([ames_train, train_poly_temp], axis=1)
ames_test_poly = pd.concat([ames_test, test_poly_temp], axis=1)

print(ames_train.shape, ames_test.shape)
print(train_poly_temp.shape, test_poly_temp.shape)
print(ames_train_poly.shape, ames_test_poly.shape)

In [ ]:
print(ames_train_poly.shape)
all_columns = "+".join(ames_train_poly.columns.difference(["LogSalePrice"]))
my_formula = "LogSalePrice~" + all_columns +'-1'
print(my_formula)

mod_naive_poly = smf.ols(my_formula, data=ames_train_poly)
nlr_poly = mod_naive_poly.fit()

print(nlr_poly.summary())

In [ ]:
print_metrics(nlr_poly, ames_train_poly, y_train, ames_test_poly, y_test, flag_log_sale_price = True)
print_metrics(nlr_poly, ames_train_poly, y_train, ames_test_poly, y_test, flag_log_sale_price = False)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

## Tuning parameters for LASSO

In [ ]:
X_train_poly = ames_train_poly.drop(columns='LogSalePrice')
X_test_poly = ames_test_poly.drop(columns='LogSalePrice')

X_train_poly_wide = pd.get_dummies(X_train_poly)
X_test_poly_wide = pd.get_dummies(X_test_poly)

In [ ]:
X_train_lasso = X_train_poly_wide
X_test_lasso = X_test_poly_wide

print(X_train_lasso.shape, X_test_lasso.shape)

In [ ]:
alpha = 0.1
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
alpha = 1e-2
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
alpha = 1e-3
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
alpha = 1e-4
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
alpha = 1e-5
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
alpha = 1e-6
lasso = Lasso(alpha=alpha, random_state=88)
lasso.fit(X_train_lasso, y_train)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

In [ ]:
MAE_list = []
candidate_values = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

for alpha in candidate_values:
    lasso = Lasso(alpha=alpha, random_state=88)
    lasso.fit(X_train_lasso, y_train)
    y_pred_test = pd.Series(lasso.predict(X_test_lasso)).apply(np.exp).reset_index(drop=True)
    y_train_exp = y_train.copy().apply(np.exp).reset_index(drop=True)
    y_test_exp = y_test.copy().apply(np.exp).reset_index(drop=True)
    MAE_list.append(MAE(y_test_exp, y_pred_test))

In [ ]:
plt.plot(candidate_values, MAE_list,'o-', markersize = 5)
plt.xlabel('Value of lambda')
plt.ylabel('OSR2')
plt.xscale('log') 

## In-class activity 1: Create a similar plot for Ridge regression. The candidate value for labmda is `[ 1e-1, 1, 10, 1e2, 1e3, 1e4 ]`. Y-axis is the OSR2 and X-axis is the value of lambda.

Add Training set performance to the graph.

In [ ]:
TrainingMAE_list = []
candidate_values = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

for alpha in candidate_values:
    lasso = Lasso(alpha=alpha, random_state=88)
    lasso.fit(X_train_lasso, y_train)
    y_pred_train = pd.Series(lasso.predict(X_train_lasso)).apply(np.exp).reset_index(drop=True)
    y_train_exp = y_train.copy().apply(np.exp).reset_index(drop=True)
    y_test_exp = y_test.copy().apply(np.exp).reset_index(drop=True)
    TrainingMAE_list.append(MAE(y_train_exp, y_pred_train))

In [ ]:
plt.plot(candidate_values, MAE_list,'o-', markersize = 5, label = 'Out-of-sample')
plt.plot(candidate_values, TrainingMAE_list,'o-', markersize = 5, label = 'In-sample')
plt.xlabel('Value of lambda')
plt.ylabel('')
plt.xscale('log') 
plt.legend();

### K-fold cross validation

In [ ]:
from sklearn.model_selection import GridSearchCV


alpha_grid = {'alpha': np.logspace(-8, -1, num=10, base=10)}

lasso_cv = GridSearchCV(lasso, param_grid = alpha_grid, scoring='neg_mean_squared_error', cv=10, verbose=1)
lasso_cv.fit(X_train_lasso, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

def one_standard_error_rule(model, results, param_grid, n_splits, neg_mean_squared_error=True):
    
    assert neg_mean_squared_error == True # function is defined specifically for neg_mean_squared_error
    
    range_x = param_grid # results['param_'+list(param_grid.keys())[0]].data
    std_vs_x  = pd.Series(results['std_test_score'], index = range_x)
    sem_vs_x  = std_vs_x/np.sqrt(n_splits)
    
    mean_vs_x = pd.Series(results['mean_test_score'], index = range_x)        
    mean_vs_x = mean_vs_x*(-1)
    
    x_min = mean_vs_x.idxmin()
    sem = sem_vs_x[x_min]
    
    if (model=='pcr'):
        x_1se = mean_vs_x[mean_vs_x <= min(mean_vs_x) + sem].index.min()
    elif (model=='ridge') | (model=='lasso'):
        x_1se = mean_vs_x[mean_vs_x <= min(mean_vs_x) + sem].index.max()
        
    #x_1se_idx = int(np.argwhere(range_x == x_1se)[0])
    
    return x_min, x_1se

In [ ]:
range_alpha = lasso_cv.cv_results_['param_alpha'].data
MSE_scores = lasso_cv.cv_results_['mean_test_score']*(-1)
x_min, x_1se = one_standard_error_rule(model='lasso',
                                       results=lasso_cv.cv_results_,
                                       param_grid=range_alpha,
                                       n_splits=10,
                                       neg_mean_squared_error=True)
plt.figure(figsize=(8, 6))
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel('Alpha', fontsize=16)
plt.ylabel('CV MSE', fontsize=16)
plt.scatter(range_alpha, MSE_scores, s=30)
plt.plot(range_alpha, MSE_scores)
plt.axvline(x=x_min, color='m')
plt.axvline(x=x_1se, color='c')
plt.grid(True, which='both')

plt.tight_layout()
plt.show()

Magenta vertical line is the minimizer, the cyan vertical line is the "1 Standard Error" selection.

In [ ]:
acc = lasso_cv.cv_results_['mean_test_score'] # what sklearn calls mean_test_score is the holdout set, i.e. the validation set.
ccp = lasso_cv.cv_results_['param_alpha'].data

pd.DataFrame({'ccp alpha' : ccp, 'Validation Accuracy': acc})

In [ ]:
print('Alpha one standard error rule:', x_1se)

### Lasso Refit with One Standard Error Rule

In [ ]:
lasso_cv = GridSearchCV(lasso, {'alpha': [x_1se]}, scoring='neg_mean_squared_error', cv=10)
lasso_cv.fit(X_train_lasso, y_train)

print_metrics(lasso_cv, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
#print_metrics(lasso_cv, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)

## Shuffle the dataset for k-fold cross validation

In [ ]:
from sklearn.model_selection import KFold

alpha_grid = {'alpha': np.logspace(-8, -1, num=15, base=10)}
cv = KFold(n_splits = 10, random_state = 1, shuffle = True)
lasso_cv = GridSearchCV(lasso, param_grid = alpha_grid, scoring='neg_mean_squared_error', cv=cv, verbose=2)
lasso_cv.fit(X_train_lasso, y_train)

In [ ]:
range_alpha = lasso_cv.cv_results_['param_alpha'].data
MSE_scores = lasso_cv.cv_results_['mean_test_score']*(-1)
x_min, x_1se = one_standard_error_rule(model='lasso',
                                       results=lasso_cv.cv_results_,
                                       param_grid=range_alpha,
                                       n_splits=10,
                                       neg_mean_squared_error=True)
plt.figure(figsize=(8, 6))
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel('Alpha', fontsize=16)
plt.ylabel('CV MSE', fontsize=16)
plt.scatter(range_alpha, MSE_scores, s=30)
plt.plot(range_alpha, MSE_scores)
plt.axvline(x=x_min, color='m')
plt.axvline(x=x_1se, color='c')
plt.grid(True, which='both')

plt.tight_layout()
plt.show()

## Custom loss function

In [ ]:
def large_prediction_error_count(y_test, y_pred, threshold = 2000):
    y_pred_test = pd.Series(y_pred).copy().apply(np.exp).reset_index(drop=True)
    y_test_exp = pd.Series(y_test).copy().apply(np.exp).reset_index(drop=True)
    count = [(y_pred_test - y_test_exp) > 2000]
    return np.sum(count)

In [ ]:
from sklearn.metrics import make_scorer  

alpha_grid = {'alpha': np.logspace(-8, -1, num=10, base=10)}
cv = KFold(n_splits = 10, random_state = 1, shuffle = True)
lasso_cv = GridSearchCV(lasso, param_grid = alpha_grid, scoring=make_scorer(large_prediction_error_count, greater_is_better=False), cv=cv, verbose=2)
lasso_cv.fit(X_train_lasso, y_train)

In [ ]:
range_alpha = lasso_cv.cv_results_['param_alpha'].data
new_scores = lasso_cv.cv_results_['mean_test_score']*(-1)

plt.figure(figsize=(8, 6))
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel('Alpha', fontsize=16)
plt.ylabel('Customized loss', fontsize=16)
plt.scatter(range_alpha, new_scores, s=30)
plt.plot(range_alpha, new_scores)
plt.grid(True, which='both')

plt.tight_layout()
plt.show()

## Cross validation for Principal Components Regression

In [ ]:
y_train = ames_train['LogSalePrice']
y_test = ames_test['LogSalePrice']

X_train_pcr = X_train_poly_wide
X_test_pcr = X_test_poly_wide

print(X_train_poly_wide.shape, X_train_pcr.shape)
print(X_test_poly_wide.shape, X_test_pcr.shape)

We also standardize the data before feeding it to the PCA step, as recommended by good practice.

In [ ]:
from sklearn.pipeline import Pipeline
scaler = StandardScaler()
pca = PCA(random_state=88)
lr = LinearRegression()
pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('lr', lr)])

Basic PCR

In [ ]:
pipe.set_params(pca__n_components=5)
pipe.fit(X_train_pcr, y_train)
print_metrics(pipe, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = True)
print_metrics(pipe, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = False)

# In-class activity 2: For PCR, do a 5 fold cross validation value for `n_components` in terms of the `R-squared`. The potential  `n_components` is between 1 and 300. 
- What are the best R2 value and its corresponding n_components? 
- What is the value of `n_components` according to the one standard error rule?
- Refit the model using the `n_components` selected by the one standard error rule.

In [ ]:
param_grid = {'pca__n_components': np.linspace(1, 300, 35).astype('int')}

pcr_cv = GridSearchCV(pipe,
                      param_grid,
                      scoring='r2',
                      cv=5,
                     verbose = 2)
pcr_cv.fit(X_train_pcr, y_train)

In [ ]:
from scipy import stats

n_components = pcr_cv.cv_results_['param_pca__n_components'].data
R2_scores = pcr_cv.cv_results_['mean_test_score']
x_min, x_1se = one_standard_error_rule(model='pcr',
                                       results=pcr_cv.cv_results_,
                                       param_grid=n_components,
                                       n_splits=10,
                                       neg_mean_squared_error=True)

plt.figure(figsize=(8, 6))
plt.xlabel('n components', fontsize=16)
plt.ylabel('CV R2', fontsize=16)
plt.scatter(n_components, R2_scores, s=30)
plt.axvline(x=x_min, color='m')
plt.axvline(x=x_1se, color='c')
plt.grid(True, which='both')

plt.tight_layout()
plt.show()

In [ ]:
best_r2_score = pcr_cv.best_score_
print("Best R2 Score from Cross-Validation:", best_r2_score)
best_n_components = pcr_cv.best_params_['pca__n_components']
print("Best n_components value:", best_n_components)

In [ ]:
print('pca n_components', x_1se)
index_x_1se = np.where(n_components == x_1se)[0][0]

# Get the corresponding R2 score
R2_score_x_1se = R2_scores[index_x_1se]
print("R2 Score corresponding to x_1se:", R2_score_x_1se)

In [ ]:
print_metrics(pcr_cv, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = True)
# print_metrics(pcr_cv, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = False)

#### Refit the model with the selected parameter

In [ ]:
pipe.set_params(pca__n_components=x_1se)
pipe.fit(X_train_lasso, y_train)
pipe.get_params()

In [ ]:
print_metrics(pipe, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = True)
print_metrics(pipe, X_train_pcr, y_train, X_test_pcr, y_test, flag_log_sale_price = False)

## Cross validation for Ridge Regression

We can choose `alpha_max` so as the value that makes all coefficientes zero, and then construct a log sequence of `alpha` values trending smaller, decreasing the degree of regularization. 

For the case of `Ridge` Regression, alpha value that would make all coefficients zero would be `Inf`, however we can be satisfied with sufficiently small numbers, and work from there.

In [ ]:
X_train_rr = X_train_poly_wide
X_test_rr = X_test_poly_wide

print(X_train_rr.shape, X_test_rr.shape)

### Determine 'alpha_max'

In [ ]:
from sklearn.linear_model import Ridge

alpha_max = 10**5
rr = Ridge(alpha=alpha_max, random_state=88)
rr.fit(X_train_rr, y_train)

### Ridge Hyper-parameter Tuning

In [ ]:
alpha_grid = {'alpha': np.logspace(-1, 5, num=50, base=10)}

rr = Ridge(random_state=88)
rr_cv = GridSearchCV(rr, alpha_grid, scoring='neg_mean_squared_error', cv=5)
rr_cv.fit(X_train_rr, y_train)

In [ ]:
range_alpha = rr_cv.cv_results_['param_alpha'].data
MSE_scores = rr_cv.cv_results_['mean_test_score']*(-1)
x_min, x_1se = one_standard_error_rule(model='ridge',
                                       results=rr_cv.cv_results_,
                                       param_grid=range_alpha,
                                       n_splits=10,
                                       neg_mean_squared_error=True)
plt.figure(figsize=(8, 6))
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel('Alpha', fontsize=16)
plt.ylabel('CV MSE', fontsize=16)
plt.scatter(range_alpha, MSE_scores, s=30)
plt.axvline(x=x_min, color='m')
plt.axvline(x=x_1se, color='c')
plt.grid(True, which='both')

plt.tight_layout()
plt.show()

In [ ]:
print('Alpha one standard error rule:', x_1se)

### Ridge Refit with One Standard Error Rule

In [ ]:
lasso.set_params(alpha=x_1se)
lasso.fit(X_train_lasso, y_train)
lasso.get_params()

In [ ]:
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = True)
print_metrics(lasso, X_train_lasso, y_train, X_test_lasso, y_test, flag_log_sale_price = False)